<H1> To compare the famous cities of the world as tourist destination based on the venues nearby. 

Import the libraries which are required

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
%matplotlib inline 
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
mpl.style.use('ggplot') # optional: for ggplot-like style

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

<H4> Only following cities will be compared

In [2]:
places = ['Paris','London','Sydney','New York','Toronto','Wellington']

In [3]:
column_names = ['Cites', 'Latitude', 'Longitude']
# instantiate the dataframe
city_df = pd.DataFrame(columns=column_names)

In [5]:
for place in places:
    address = place
    geolocator = Nominatim(user_agent="my-application")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinate of'+ place +' are {}, {}.'.format(latitude, longitude))
    city_df = city_df.append({'Cites':place,'Latitude':latitude,'Longitude':longitude},ignore_index=True)
    
city_df.head()

The geograpical coordinate ofParis are 48.8566101, 2.3514992.
The geograpical coordinate ofLondon are 51.5073219, -0.1276474.
The geograpical coordinate ofSydney are -33.8548157, 151.2164539.
The geograpical coordinate ofNew York are 40.7127281, -74.0060152.
The geograpical coordinate ofToronto are 43.653963, -79.387207.
The geograpical coordinate ofWellington are -41.2887953, 174.7772114.


,Cites,Latitude,Longitude
0,Paris,48.856610,2.351499
1,London,51.507322,-0.127647
2,Sydney,-33.854816,151.216454
3,New York,40.712728,-74.006015
4,Toronto,43.653963,-79.387207


<H4> Foursquare Credentials to get top 10 venues of the city.

In [6]:
CLIENT_ID = 'IXZTV3FGO0IWSZU40ZJZUWVSAMKKVPW2PZ32A14EREGUG3ZH' #your Foursquare ID
CLIENT_SECRET ='TU3G3DBF5PEO5N5YAZMJEIMKDUDEBK0QVADP1PF21KXJCBCA' # your Foursquare Secret
VERSION = '20191908'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IXZTV3FGO0IWSZU40ZJZUWVSAMKKVPW2PZ32A14EREGUG3ZH
CLIENT_SECRET:TU3G3DBF5PEO5N5YAZMJEIMKDUDEBK0QVADP1PF21KXJCBCA


In [7]:
def Top10Venues(names, latitudes, longitudes, radius=500, LIMIT=10):
    
    venues=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues.append([(
            name, 
            lat, 
            lng,            
            v['venue']['name'], 
            v['venue']['id'],
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues for item in venue_list])
    nearby_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue',
                  'Venue id',                
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category'        
                   ]
    
    return(nearby_venues)

<H2> The dataset which will be used for comparision

In [8]:
city_venues = Top10Venues(names=city_df['Cites'], latitudes=city_df['Latitude'], longitudes=city_df['Longitude'])
city_venues

Paris
London
Sydney
New York
Toronto
Wellington


,City,City Latitude,City Longitude,Venue,Venue id,Venue Latitude,Venue Longitude,Venue Category
0,Paris,48.856610,2.351499,Place de l'Hôtel de Ville – Esplanade de la Li...,4bf41231e5eba59334341f90,48.856925,2.351412,Plaza
1,Paris,48.856610,2.351499,Parc Rives de Seine,58e0f18f561ded7abcbbbd01,48.855510,2.351419,Park
2,Paris,48.856610,2.351499,Berges de Seine – Rive droite,58d800df9435a979b8a645fa,48.855131,2.352289,Pedestrian Plaza
3,Paris,48.856610,2.351499,L'Alsacien,54351b59498e123ee4afc7ff,48.858275,2.350381,Alsatian Restaurant
4,Paris,48.856610,2.351499,Häagen-Dazs,4cb2fc20db32f04d6da9d34d,48.854879,2.350233,Ice Cream Shop
5,Paris,48.856610,2.351499,LUSH,4d628e33e4fe548152b8979e,48.857791,2.351622,Cosmetics Shop
6,Paris,48.856610,2.351499,BHV Marais,4b4b6a3df964a520b89a26e3,48.857312,2.353633,Department Store
7,Paris,48.856610,2.351499,Maison Aleph,59247f4e0d2be70573edd834,48.857348,2.354873,Pastry Shop
8,Paris,48.856610,2.351499,Square de la Tour Saint-Jacques,4dbd336b6a23e294ba405cfa,48.857882,2.348757,Park
9,Paris,48.856610,2.351499,Tour Saint-Jacques,4b5c7d1ff964a5205f3229e3,48.858031,2.348875,Historic Site


<H4> Lets check some details about the venues

In [9]:
city_view = city_venues.groupby('City').count()
city_cat = city_view['Venue Category']
city_cat

City
London        10
New York      10
Paris         10
Sydney        10
Toronto       10
Wellington    10
Name: Venue Category, dtype: int64

In [10]:
print('There are {} uniques categories.'.format(len(city_venues['Venue Category'].unique())))
print(city_venues['Venue Category'].unique() )

There are 45 uniques categories.
['Plaza' 'Park' 'Pedestrian Plaza' 'Alsatian Restaurant' 'Ice Cream Shop'
 'Cosmetics Shop' 'Department Store' 'Pastry Shop' 'Historic Site'
 'Art Museum' 'Fountain' 'Spa' 'Church' 'Hotel' 'Monument / Landmark'
 'Art Gallery' 'Outdoor Sculpture' 'Concert Hall' 'Theater' 'Opera House'
 'Australian Restaurant' 'Café' 'Cocktail Bar' 'Hotel Bar'
 'Laundry Service' 'Coffee Shop' 'Dance Studio' 'French Restaurant'
 'Gym / Fitness Center' 'Bakery' 'Indian Restaurant' 'Neighborhood'
 'Sushi Restaurant' 'Poke Place' 'Ramen Restaurant' 'Tea Room'
 'Japanese Restaurant' 'Arts & Crafts Store' 'Bubble Tea Shop' 'Waterfront'
 'Creperie' 'Burger Joint' 'Gastropub' 'Vietnamese Restaurant' 'Beer Bar']


In [11]:
city_venues.groupby('Venue Category').count()

,City,City Latitude,City Longitude,Venue,Venue id,Venue Latitude,Venue Longitude
Venue Category,,,,,,,
Alsatian Restaurant,1,1,1,1,1,1,1
Art Gallery,2,2,2,2,2,2,2
Art Museum,3,3,3,3,3,3,3
Arts & Crafts Store,1,1,1,1,1,1,1
Australian Restaurant,1,1,1,1,1,1,1
Bakery,1,1,1,1,1,1,1
Beer Bar,1,1,1,1,1,1,1
Bubble Tea Shop,1,1,1,1,1,1,1
Burger Joint,1,1,1,1,1,1,1


<H2> Preparing data for clustering and clusturing them using K-Means

In [12]:
# one hot encoding
city_onehot = pd.get_dummies(city_venues[['Venue Category']], prefix="", prefix_sep="")

# add Lieux column back to dataframe
city_onehot['City'] = city_venues['City'] 

# move lieux column to the first column
fixed_columns = [city_onehot.columns[-1]] + list(city_onehot.columns[:-1])
city_onehot = city_onehot[fixed_columns]

city_onehot.head()

,City,Alsatian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Australian Restaurant,Bakery,Beer Bar,Bubble Tea Shop,Burger Joint,...,Pedestrian Plaza,Plaza,Poke Place,Ramen Restaurant,Spa,Sushi Restaurant,Tea Room,Theater,Vietnamese Restaurant,Waterfront
0,Paris,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,Paris,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Paris,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,Paris,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Paris,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
city_grouped = city_onehot.groupby('City').mean().reset_index()
city_grouped

,City,Alsatian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Australian Restaurant,Bakery,Beer Bar,Bubble Tea Shop,Burger Joint,...,Pedestrian Plaza,Plaza,Poke Place,Ramen Restaurant,Spa,Sushi Restaurant,Tea Room,Theater,Vietnamese Restaurant,Waterfront
0,London,0.0,0.1,0.2,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.1,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0
1,New York,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Paris,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.1,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Sydney,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,...,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.3,0.0,0.0
4,Toronto,0.0,0.0,0.1,0.1,0.0,0.0,0.0,0.1,0.0,...,0.0,0.1,0.1,0.1,0.0,0.1,0.1,0.0,0.0,0.0
5,Wellington,0.0,0.1,0.0,0.0,0.0,0.0,0.1,0.0,0.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.1


In [14]:
kclusters = 3

city_clustering = city_grouped.drop('City', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(city_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 2, 2, 1, 0, 0])

In [15]:
#Let's create a new dataframe that includes the cluster as well as the top 5 venues for each Lieux.

city_merged = city_df

# add clustering labels
city_merged['Cluster Labels'] = kmeans.labels_


city_merged 
# check the last columns!

,Cites,Latitude,Longitude,Cluster Labels
0,Paris,48.856610,2.351499,0
1,London,51.507322,-0.127647,2
2,Sydney,-33.854816,151.216454,2
3,New York,40.712728,-74.006015,1
4,Toronto,43.653963,-79.387207,0
5,Wellington,-41.288795,174.777211,0


<H2> Creating Map to show the result of clustering

In [16]:
latitude = city_merged.iloc[1]['Latitude']
longitude = city_merged.iloc[1]['Longitude']
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=1)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(city_merged['Latitude'], city_merged['Longitude'], city_merged['Cites'], city_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters